<span style="font-family:Lucida Bright;">
<p style="margin-bottom:0.5cm"></p>
<center>
<font size="8"><b>Deep Learning, Fall 2021</b></font>
<p style="margin-bottom:0.6cm"></p>
<font size="3"><b>Final Project:</b></font>
<p style="margin-bottom:0.6cm"></p>
<font size="5"><b>Enhancing Voices for Better Speech Intelligibility</b></font>
<p style="margin-bottom:2cm"></p>
<font size="6"><b>Start</b></font>
</center>
<p style="margin-bottom:2cm"></p>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#How-to-get-the-most-out-of-this-notebook" data-toc-modified-id="How-to-get-the-most-out-of-this-notebook-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>How to get the most out of this notebook</a></span></li><li><span><a href="#Prerequisites" data-toc-modified-id="Prerequisites-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Prerequisites</a></span></li><li><span><a href="#Links-and-resources" data-toc-modified-id="Links-and-resources-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Links and resources</a></span></li></ul></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Initialization" data-toc-modified-id="Initialization-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Initialization</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Load data</a></span><ul class="toc-item"><li><span><a href="#TIMIT-dataset" data-toc-modified-id="TIMIT-dataset-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>TIMIT dataset</a></span></li><li><span><a href="#Synthetic-speech-dataset" data-toc-modified-id="Synthetic-speech-dataset-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Synthetic speech dataset</a></span></li></ul></li></ul></li></ul></div>

# Introduction

Many people struggle to understand speech in challenging acoustic environments, such as noisy bar. Therefore, enhancing the intelligibility of noisy speech signals is one of the key challenges for any producer of modern communication devices.

The problem is often tackled by dividing a noisy speech signal into a number of frequency bands and attenuating the ones where the signal-to-noise ratio is insufficient. This approach, while effective in some situations, often leads to poor results, and sometimes even exacerbates the problem it is trying to solve as the constant activation and disactivation of some of the frequency bands in response to the fluctuations in speech and noise can create a very unnatural and disturbing sounds.

In this project, we will try a different approach and attempt to create a deep learning model that will produce an equalization curve that can be applied to the noisy speech signal in order to maximize its intelligibility.


## Official project description

In the info-doc for the Deep Learning course, the project is described as follows:

> *__Designing self-driving earbuds with [augmentedhearing.io](augmentedhearing.io) which enhance voices based on function correlated with speech intelligibility__*
>
> *As one in four adults struggle to understand speech in challenging acoustics we aim to train consumer earbuds to enhance voices through back propagation using DHASP model implemented using [PyTorch differentiation package](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9414571)* validated on [TIMIT speech dataset](https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3) based on an objective function correlated with [HASPI speech intelligibility auditory processing model](https://www.sciencedirect.com/science/article/pii/S0167639320300431) available in Matlab.*

## The approach

The time scope for the course is 7 days x 9 hours / day = 63 hours. The outcome should be documented in a report formatted as [conference paper](https://drive.google.com/file/d/0BxJRy96AHCJxaUEwOFhwUExmX00/view?usp=sharing&resourcekey=0-RvwJqDVrZVijbkkifLWoYA), as well as a Jupyter notebook that ideally should recreate the main results of the report.

At the beginning of the project, the following was available:

1. speech data: a TIMIT dataset containing of 10 sentences spoken by 630 speakers from 8 major dialect regions of the United States.
2. an [article](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9414571) describing a proposal for a differentiable objective function that can be used to train a neural network. The function is a measure of similarity of a [cepstrum](https://en.wikipedia.org/wiki/Cepstrum) of a given speech signal to the cepstrum of the target signal (usually clear, noisless speech)
3. Matlab code for calculating the [HASPI speech intelligibility index] that can be used to evaluate the results.

To create a our model, the following is needed:

1. clear speech audio data to use as the target for model
2. the corresponding noisy speech audio data to train the mode
3. a PyTorch implementation of the equalization filter that can be applied to the speech signal. Our model will optimize the parameters of this filter to maximize speech intelligibility .
4. a working PyTorch implementation of the objective function proposed in the [article](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9414571).

Therefore, before we even create our neural network, we need to obtain the prerequisites 2 - 4, out of which especially number 4: implementation of the objective function proposed in the [article](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9414571) is, as I will demonstrate later, all but straightforward. To go through and understand the project literature, obtain the prerequisites, create and optimize model, and document the findings is a task that extends way beyond the 63-hour scope of this course. The work presented in this notebook is a result more then 3 times as much mork, and contains a functioning implementation of the all the prerequisites necessary to build a neural network that will attempt to optimize our objective.


















## How to get the most out of this notebook

To fully enjoy the content of this notebook, please take note of the following:

- The outputs of pre-executed cells will not be rendered properly unless the notebook is **Trusted**.

- To avoid accidental changes, most of the cells in this notebook are marked-as read only, and many are frozen (i.e. disabled from being run). To take advantage of these features, it is recommended to use the extension
[Freeze Cell](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/freeze/readme.html)  which works with Jupyter Notebook.

## Prerequisites

This notebook is a part of project. To be able to execute all of its content:

1. Download the project repository: `https://github.com/wojciechdk/deep-learning-2021-final-project.git` and run the notebook from the root.

2. Download the [TIMIT dataset](https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3), unpack it, and place it so that the folders *DOC*, *TEST*, and *TRAIN* are placed in the folder `[root]\resources\data\TIMIT`.

## Links and resources

- If you are reading the HTML version of this notebook and would like to download the ipynb source, you can do it [here](https://github.com/wojciechdk/deep-learning-2021-final-project/blob/main/main.ipynb).

- The Git repository containing the entire project can be accessed [here](https://github.com/wojciechdk/deep-learning-2021-final-project.git).


# Setup

## Initialization

The entire initialization process, including:
 - imports of the necessary packages
 - configuration of the notebook and packages
 - imports of the toolbox functions

is defined in the file [toolbox/initialization.py](toolbox/initialization.py)

In [1]:
from toolbox.initialization import *

## Load data

### TIMIT dataset

The metadata about the TIMIT dataset is loaded using the function `load_timit_data` defined in the module `toolbox\data_loading.py`. Once loaded, the data is saved in the project cache as a Pandas dataframe. Let's load it and show a couple of rows:

In [2]:
# Load the TIMIT meta data from the cache.
df_timit = pd.read_pickle(paths.cache.df_timit)

# Show the top 5 rows
display(df_timit.sample(5))

,sentence_number,data_group,dialect,gender,speaker,type,text,audio_path,start_sample,end_sample,words_text,words_start_sample,words_end_sample,phonemes_text,phonemes_start_sample,phonemes_end_sample
3723,1681,train,Western,F,LEH0,phonetically-diverse,The sheriff's swivel chair tilted back.,train\DR7\FLEH0\SI1681.wav,0,38503,"[the, sheriff's, swivel, chair, tilted, back]","[2308, 3360, 13066, 16793, 22057, 29490]","[3360, 12600, 16793, 22057, 29490, 35550]","[h#, dh, ix, sh, eh, r, ix, f, s, epi, w, ih, ...","[0, 2308, 2533, 3360, 6040, 7644, 8605, 9242, ...","[2308, 2533, 3360, 6040, 7644, 8605, 9242, 100..."
1968,435,train,South Midland,F,JWB1,phonetically-compact,The rich should invest in black zircons instea...,train\DR4\FJWB1\SX435.wav,0,54375,"[the, rich, should, invest, in, black, zircons...","[2140, 3160, 6340, 9942, 16420, 17754, 22050, ...","[3160, 6340, 9942, 16420, 17754, 22050, 30590,...","[h#, dh, ax, r, ih, tcl, sh, uh, dx, uh, n, v,...","[0, 2140, 2260, 3160, 4386, 5890, 6340, 8760, ...","[2140, 2260, 3160, 4386, 5890, 6340, 8760, 973..."
2088,439,train,South Midland,M,BWP0,phonetically-compact,That stinging vapor was caused by chloride vap...,train\DR4\MBWP0\SX439.wav,0,54989,"[that, stinging, vapor, was, caused, by, chlor...","[2230, 5390, 13280, 19258, 23101, 30258, 32360...","[5390, 13280, 19258, 23101, 30258, 32360, 4084...","[h#, dh, ae, tcl, s, tcl, t, ih, ng, iy, ng, v...","[0, 2230, 2715, 4440, 5390, 6983, 7780, 8149, ...","[2230, 2715, 4440, 5390, 6983, 7780, 8149, 939..."
4345,164,train,Western,M,TMN0,phonetically-compact,Curiosity and mediocrity seldom coexist.,train\DR7\MTMN0\SX164.wav,0,56730,"[curiosity, and, mediocrity, seldom, coexist]","[3210, 17002, 19091, 33073, 40098]","[17002, 19091, 33073, 40098, 53255]","[h#, k, y, er, iy, aa, s, ix, dx, iy, ih, n, m...","[0, 3210, 4660, 5389, 7352, 8689, 10986, 12814...","[3210, 4660, 5389, 7352, 8689, 10986, 12814, 1..."
4524,937,train,Army Brat,M,KDD0,phonetically-diverse,Let the orthodontist decide the proper time to...,train\DR8\MKDD0\SI937.wav,0,66253,"[let, the, orthodontist, decide, the, proper, ...","[1532, 6330, 8123, 20845, 28350, 29240, 35080,...","[6330, 8123, 20845, 28350, 29240, 35080, 40674...","[h#, l, eh, tcl, dh, iy, ao, r, th, ix, dcl, d...","[0, 1532, 3020, 4680, 6330, 6710, 8123, 9274, ...","[1532, 3020, 4680, 6330, 6710, 8123, 9274, 101..."


Now, let's play one sentence from the dataset:

In [3]:
t.sound.play_timit(df_timit.loc[0, :])

### Synthetic speech dataset

The metadata about the dataset containing synthetic speech in different audio settings is loaded using the function `load_synthetic_speech_data` defined in the module `toolbox\data_loading.py`. Once loaded, the data is saved in the project cache as a Pandas dataframe. Let's load it and show a couple of rows:

In [4]:
# Load the meta data about the synthetic speech dataset from the cache.
df_synthetic_speech = pd.read_pickle(paths.cache.df_synthetic_speech)

# Show the top 5 rows.
display(df_synthetic_speech.sample(5))

,speaker,length,variant,segment,fullness,clarity,audio_path
2174,kevin,5s,babble,3,NaN,NaN,cut_5_s\kevin\babb\3.wav
1797,kendra,5s,babble,4,NaN,NaN,cut_5_s\kendra\babb\4.wav
3051,matthew,5s,tv,7,NaN,NaN,cut_5_s\matthew\tele\7.wav
1446,justin,5s,music,17,NaN,NaN,cut_5_s\justin\musi\17.wav
3223,salli,5s,zoom_augmented,33,1,4,cut_5_s\salli\1-4\33.wav


Let's play one of the first 5s segments from the database:

In [5]:
# Choose which file to play.
mask = (
    (df_synthetic_speech['speaker'] == 'joanna')
    & (df_synthetic_speech['length'] == '5s')
    & (df_synthetic_speech['segment'] == 10)
    & (df_synthetic_speech['variant'] == 'babble')
)

display(
    df_synthetic_speech.loc[mask, :]
)

# Play.
t.sound.play_synthetic_speech(df_synthetic_speech.loc[mask, :])

,speaker,length,variant,segment,fullness,clarity,audio_path
559,joanna,5s,babble,10,NaN,NaN,cut_5_s\joanna\babb\10.wav
